# Unitary dynamics & ΔNFR

Unitary flows generated by the coherence operator encode how ΔNFR reorganises the node without breaking normalization. Tracking the induced structural frequency drift keeps the nodal equation balanced.

## Workflow

1. Select a coherence operator and derive its unitary evolution `exp(-i·Ĉ)`.
2. Propagate a normalized state through the unitary to observe how frequency expectations shift.
3. Map the observed shift into a deterministic ΔNFR hook.
4. Execute a short operator sequence and confirm that EPI and νf reflect the ΔNFR update.

## Smoke check: coupling ΔNFR to unitary evolution

The script below computes a one-step unitary evolution, measures the frequency projection drift and reuses it as the ΔNFR increment applied to a node.

In [ ]:
import numpy as np

from tnfr.constants import DNFR_PRIMARY, EPI_PRIMARY, VF_PRIMARY
from tnfr.dynamics import set_delta_nfr_hook
from tnfr.mathematics.operators import CoherenceOperator, FrequencyOperator
from tnfr.mathematics.spaces import HilbertSpace
from tnfr.structural import Coherence, Emission, Reception, Resonance, Transition, create_nfr, run_sequence

space = HilbertSpace(dimension=2)
coherence_operator = CoherenceOperator([[0.8, 0.1], [0.1, 0.5]])
frequency_operator = FrequencyOperator([[1.1, 0.05], [0.05, 0.7]])

state = np.array([0.9, 0.3], dtype=np.complex128)
state = state / space.norm(state)

eigenvalues, eigenvectors = np.linalg.eigh(coherence_operator.matrix)
unitary = (eigenvectors * np.exp(-1j * eigenvalues)) @ eigenvectors.conj().T
next_state = unitary @ state

freq_before = frequency_operator.project_frequency(state)
freq_after = frequency_operator.project_frequency(next_state)
dnfr_increment = float(freq_after - freq_before)

G, node = create_nfr("unitary-demo", epi=0.5, vf=1.0, theta=0.0)

def delta_from_unitary(graph):
    graph.nodes[node][DNFR_PRIMARY] = dnfr_increment
    graph.nodes[node][EPI_PRIMARY] += dnfr_increment
    graph.nodes[node][VF_PRIMARY] += dnfr_increment * 0.1

set_delta_nfr_hook(G, delta_from_unitary)
run_sequence(G, node, [Emission(), Reception(), Coherence(), Resonance(), Transition()])

{
    "ΔNFR": round(G.nodes[node][DNFR_PRIMARY], 9),
    "EPI": round(G.nodes[node][EPI_PRIMARY], 9),
    "νf": round(G.nodes[node][VF_PRIMARY], 9),
    "frequency_shift": round(dnfr_increment, 9),
    "unitary_norm": round(space.norm(next_state), 9),
}
